In [15]:
%cd /Users/ben/prj/fab/_scripts

/Users/ben/prj/fab/_scripts


In [14]:
## FROM SYNC_CANS.PY
#!/usr/bin/env python

import fill_untappd
import yaml, json, sqlite3,  dateparser,os, sys
from pprint import pprint
from airtable import Airtable
import pandas as pd


def main():
    fill_untappd.run()
    conn = sqlite3.connect("untappd.db")
    df = pd.read_sql_query("select * from checkins;", conn)
    jsons  = df.json.apply(lambda j: json.loads(j))

    venues = jsons.apply(
        lambda x: 
        pd.concat([
            pd.Series(x["venue"]),
            pd.Series(x["user"])[["user_name"]],
            pd.Series(x["beer"]),
            pd.Series(x)[["checkin_id","created_at","rating_score"]]
            ])
            )

    venues_info = pd.concat([venues, 
    pd.concat([
        venues['contact'].apply(lambda x: pd.Series(x)),
        venues['location'].apply(lambda x: pd.Series(x)),
    ], axis = 1)],axis=1).set_index(["venue_slug","beer_slug"])

    venue_users = venues_info.groupby(["venue_slug","beer_slug"]).user_name.apply(','.join).rename("user_names")
    venue_rating = venues_info.groupby(["venue_slug","beer_slug"]).rating_score.sum().rename("total_rating")
    venue_counts = venues_info.groupby(["venue_slug","beer_slug"]).size().rename("n_checkins").astype(int)
    venue_last_checkin = venues_info.groupby(["venue_slug","beer_slug"]).created_at.apply(lambda x: x.apply(lambda y:dateparser.parse(y)).max()).rename("last_checkin")


    venues_info_dedup = venues_info.loc[~venues_info.index.duplicated()]
    venues_annotated = venues_info_dedup.join(pd.concat([venue_users, venue_rating, venue_counts,venue_last_checkin],axis=1))

    venues_annotated = venues_annotated.loc[ lambda x: x.venue_url != ""]
    venues_annotated = venues_annotated.loc[ venues_annotated.venue_url.notna()]
    venues_annotated = venues_annotated.loc[ lambda x: x.venue_city != ""]
    venues_annotated = venues_annotated.loc[ venues_annotated.venue_city.notna()]


    cans = pd.read_csv("../_data/cans.csv")
    cans["untappd_id"] = cans.untappd.str.extract('(\d*$)').astype(int)

    venue_beers_annotated = venues_annotated.loc[lambda x: x.venue_id.notna()]
    venue_beers_annotated = venue_beers_annotated.join(cans.set_index("untappd_id")[["code"]], on ="bid")
    venue_beers_annotated = venue_beers_annotated.rename({"code":"can_code"}, axis = "columns")
    venue_beers_annotated["venue_id"] = venue_beers_annotated["venue_id"].astype(int)
    venue_beers_annotated.n_checkins = venue_beers_annotated.n_checkins.astype(int)

    venue_beers_annotated.to_csv("../_data/venue_beers.csv")

    venues_annotated = venue_beers_annotated.reset_index()[["venue_name","venue_slug","primary_category","contact","venue_id","lat","lng","venue_city" , "venue_address","venue_country","venue_city",	"venue_country",	"venue_state",'venue_url']]
    venues_annotated.to_csv("../_data/venues.csv")

# if __name__ == "__main__":
#     os.chdir(os.path.abspath(os.path.dirname(sys.argv[0])))
#     main()

In [19]:
main()


{'client_id': 'E8847D1FA347C37248DEC90E1D21568E18FC5152', 'client_secret': '199C86117D183A9B6BCB3220098F34CFF81FD562', 'limit': 100, 'min_id': 861551896}
{'count': 0, 'items': []}


In [27]:
google_sheet_url_1 = "https://docs.google.com/spreadsheets/d/1cozZyyhcIRuwhdi5Z056_-8LoHC_YR_k-498HTS0-u8/export?format=csv#gid=400302575"
partners_report =pd.read_csv(google_sheet_url,skiprows=0)

In [35]:
google_sheet_url_2 = "https://docs.google.com/spreadsheets/d/1cozZyyhcIRuwhdi5Z056_-8LoHC_YR_k-498HTS0-u8/export?format=csv&id=1cozZyyhcIRuwhdi5Z056_-8LoHC_YR_k-498HTS0-u8&gid=1966703241"
sales_report=pd.read_csv(google_sheet_url_2,skiprows=0)

In [36]:
sales_report

,Company,Customer Num,Brand,On-Off Premise,Product Name,Case Equiv,$Vol
0,15 Walnut Tavern,584,FAB Squid Pro Quo,On Premise,FAB Squid Pro Quo DIPA 5.2,2.31,$129.00
1,15 Walnut Tavern,584,FAB Squid Pro Quo,On Premise,FAB Squid Pro Quo Sticker for Tap Handle (W),0.00,$0.00
2,711 Kittery,1958,FAB Balloon Factory,Off Premise,FAB Balloon Factory IPA 6/4/16,1.33,$68.00
3,711 Kittery,1958,FAB Squid Pro Quo,Off Premise,FAB Squid Pro Quo DIPA 6/4/16,1.33,$77.00
4,Abbey Brookline,1142,FAB Squid Pro Quo,On Premise,FAB Squid Pro Quo DIPA 6/4/16,2.67,$154.00
...,...,...,...,...,...,...,...
445,Yankee Spirits - Sturbridge,273,FAB Balloon Factory,Off Premise,FAB Balloon Factory IPA 6/4/16,1.33,$68.00
446,Yankee Spirits - Sturbridge,273,FAB Squid Pro Quo,Off Premise,FAB Squid Pro Quo DIPA 6/4/16,1.33,$77.00
447,Yankee Spirits - Swansea,1042,FAB Balloon Factory,Off Premise,FAB Balloon Factory IPA 6/4/16,1.33,$68.00
448,Yankee Spirits - Swansea,1042,FAB Squid Pro Quo,Off Premise,FAB Squid Pro Quo DIPA 6/4/16,1.33,$77.00


In [31]:
sales_report

,Company,City,Buyer,Additional Contact,Email,Salesman Name,Next steps,Protip,FEB14,Notes,Lat,Lng,Instagram?,Unnamed: 13
0,Fort Point Market,Boston,Tony Rodriguez,NaN,tonyfortpointmarket@gmail.com,Matt Kemon,NaN,NaN,NaN,NaN,42.349816,-71.048039,fortpointmarket,NaN
1,Whole Foods Dedham,Dedham,Jake Strickland,NaN,jacob.strickland@wholefoods.com,Tom Briggs,NaN,NaN,NaN,NaN,42.233035,-71.177959,wholefoodsdedham,NaN
2,Craft Beer Cellar - Westford,Westford,Andy Bajaj,Sue Bajaj,westford@craftbeercellar.com,Tom Briggs,NaN,From Tom: Andy is the primary buyer since Fall...,NaN,NaN,42.569300,-71.418430,cbc_westford,NaN
3,Whole Foods Hadley,Hadley,Michael Pietrantonio,NaN,michael.pietrantonio@wholefoods.com,Brian DiPippo,NaN,NaN,NaN,NaN,42.353120,-72.555805,"Haven't yet found IG, FB, or twitter specific ...",NaN
4,Whole Foods Ink Block,Boston,Sara Collins,NaN,Sara.Collins@wholefoods.com,Matt Kemon,NaN,NaN,NaN,NaN,42.345301,-71.062495,"Haven't yet found IG, FB, or twitter specific ...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
